In [85]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
import matplotlib.pyplot as plt

In [27]:
!wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFiles/Industrial_and_Scientific.json.gz

--2023-11-22 12:24:57--  https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFiles/Industrial_and_Scientific.json.gz
Resolving datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)... 132.239.8.30
Connecting to datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)|132.239.8.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 210409178 (201M) [application/x-gzip]
Saving to: ‘Industrial_and_Scientific.json.gz.2’

Industrial_and_Scie 100%[===================>] 200.66M  6.29MB/s    in 34s     

2023-11-22 12:25:32 (5.91 MB/s) - ‘Industrial_and_Scientific.json.gz.2’ saved [210409178/210409178]



In [28]:
!wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/metaFiles2/meta_Industrial_and_Scientific.json.gz

--2023-11-22 12:25:32--  https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/metaFiles2/meta_Industrial_and_Scientific.json.gz
Resolving datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)... 132.239.8.30
Connecting to datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)|132.239.8.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 83600786 (80M) [application/x-gzip]
Saving to: ‘meta_Industrial_and_Scientific.json.gz.2’

meta_Industrial_and 100%[===================>]  79.73M  7.03MB/s    in 12s     

2023-11-22 12:25:44 (6.81 MB/s) - ‘meta_Industrial_and_Scientific.json.gz.2’ saved [83600786/83600786]



# Data Reading

In [86]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, length, lower, split, when
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.feature import StandardScaler
from pyspark.sql.functions import when


# Create a Spark session
spark = SparkSession.builder \
    .appName("readGZ") \
    .master("local[*]") \
    .config("spark.default.parallelism", 24) \
    .config("spark.sql.shuffle.partitions", 24) \
    .getOrCreate()

In [87]:

# Read the gzipped JSON file directly into a DataFrame
df = spark.read.json("Industrial_and_Scientific.json.gz")

# Show the DataFrame to check if it's loaded correctly
df.show(10)

+----------+-----+-------+--------------------+-----------+--------------+----------------+-----+--------------------+--------------+--------+----+
|      asin|image|overall|          reviewText| reviewTime|    reviewerID|    reviewerName|style|             summary|unixReviewTime|verified|vote|
+----------+-----+-------+--------------------+-----------+--------------+----------------+-----+--------------------+--------------+--------+----+
|0176496920| NULL|    5.0|Arrived on time, ...|01 23, 2013|A3FANY5GOT5X0W|    Kelly Keyser| NULL|  Just as described!|    1358899200|    true|NULL|
|0176496920| NULL|    5.0|This device was h...| 11 5, 2012| AT6HRPPYOPHMB|       Michael C| NULL|        Great device|    1352073600|    true|NULL|
|0176496920| NULL|    4.0|Just a clicker no...|10 17, 2012| A4IX7B38LIN1E|              BH| NULL|         Pretty Good|    1350432000|    true|NULL|
|0176496920| NULL|    5.0|Great response ca...|03 29, 2017|A12Q4LR8N17AOZ|   Waterfall3500| NULL|Thank you for t

In [88]:
### load the meta data

df_meta = spark.read.json("meta_Industrial_and_Scientific.json.gz")

In [89]:
df_meta.show(10)

+--------------------+--------------------+----------+--------------------+--------------------+-----------------+--------------------+-------+--------------------+---+--------------------+--------------------+--------------------+------+--------------------+------------+--------------------+-----+--------------------+
|            also_buy|           also_view|      asin|               brand|            category|             date|         description|details|             feature|fit|            imageURL|     imageURLHighRes|            main_cat| price|                rank|similar_item|               tech1|tech2|               title|
+--------------------+--------------------+----------+--------------------+--------------------+-----------------+--------------------+-------+--------------------+---+--------------------+--------------------+--------------------+------+--------------------+------------+--------------------+-----+--------------------+
|[1454894547, 0133...|               

## Discussion



1.   We can not read some categories (especially those with large data, in this case, the sofrware category) directly in Spark; Need to figure out why and how to solve this. (Tried reading by in raw and use pandas and then spark, but this also failed.)
(I skipped this question by choosing another category which is fine to read directly)





# Data Pre-Processing and visulization

In [90]:
# summarize the nulls in dataframe
for column in df.columns:
    # Calculate the number of nulls in the column
    null_count = df.filter(col(column).isNull()).count()
    
    # Calculate the percentage of nulls
    null_percentage = (null_count / df.count()) * 100
    
    # Print the results
    print(f"Number of Nulls in {column}: {null_count}, Percentage: {null_percentage:.2f}%")


Number of Nulls in asin: 0, Percentage: 0.00%


Number of Nulls in image: 1725623, Percentage: 98.14%


Number of Nulls in overall: 0, Percentage: 0.00%


Number of Nulls in reviewText: 984, Percentage: 0.06%


Number of Nulls in reviewTime: 0, Percentage: 0.00%


...

Number of Nulls in style: 1066819, Percentage: 60.67%


Number of Nulls in summary: 403, Percentage: 0.02%


Number of Nulls in unixReviewTime: 0, Percentage: 0.00%


Number of Nulls in verified: 0, Percentage: 0.00%


Number of Nulls in vote: 1552025, Percentage: 88.27%


In [91]:
# Drop the image, style, vote columns
df = df.drop('image', 'style', 'vote')


In [92]:
from pyspark.sql.functions import col, when, length, unix_timestamp, to_date

# Handling missing values: For simplicity, we'll drop rows with any NULLs
df = df.na.drop()

# Filtering out unverified reviews
df = df.filter(col('verified') == True)

# Feature Engineering - creating a new feature for the length of the reviewText
df = df.withColumn('reviewText_length', length(col('reviewText')))


In [93]:
df.count()

1635211

In [94]:
# Show the processed DataFrame
df.show(10)

+----------+-------+--------------------+-----------+--------------+----------------+--------------------+--------------+--------+-----------------+
|      asin|overall|          reviewText| reviewTime|    reviewerID|    reviewerName|             summary|unixReviewTime|verified|reviewText_length|
+----------+-------+--------------------+-----------+--------------+----------------+--------------------+--------------+--------+-----------------+
|0176496920|    5.0|Arrived on time, ...|01 23, 2013|A3FANY5GOT5X0W|    Kelly Keyser|  Just as described!|    1358899200|    true|              121|
|0176496920|    5.0|This device was h...| 11 5, 2012| AT6HRPPYOPHMB|       Michael C|        Great device|    1352073600|    true|              201|
|0176496920|    4.0|Just a clicker no...|10 17, 2012| A4IX7B38LIN1E|              BH|         Pretty Good|    1350432000|    true|              167|
|0176496920|    5.0|Great response ca...|03 29, 2017|A12Q4LR8N17AOZ|   Waterfall3500|Thank you for the...|

In [95]:
review_lengths = df.select('reviewText_length').rdd.flatMap(lambda x: x).collect()

23/11/22 13:27:44 ERROR RetryingBlockTransferor: Exception while beginning fetch of 1 outstanding blocks 
java.io.IOException: Failed to connect to /172.20.10.3:55898
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:294)
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:214)
	at org.apache.spark.network.netty.NettyBlockTransferService$$anon$2.createAndStart(NettyBlockTransferService.scala:131)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.transferAllOutstanding(RetryingBlockTransferor.java:173)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.start(RetryingBlockTransferor.java:152)
	at org.apache.spark.network.netty.NettyBlockTransferService.fetchBlocks(NettyBlockTransferService.scala:151)
	at org.apache.spark.network.BlockTransferService.fetchBlockSync(BlockTransferService.scala:102)
	at org.apache.spark.storage.BlockManager.fetchRemoteManagedBuffer(BlockMana

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 306.0 failed 1 times, most recent failure: Lost task 0.0 in stage 306.0 (TID 210) (172.20.10.3 executor driver): TaskResultLost (result lost from block manager)
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2844)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2780)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2779)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2779)
	at org.apache.spark.scheduler.DAGSche

In [50]:
review_lengths = df.select('reviewText_length').rdd.flatMap(lambda x: x).collect()

# Now convert this list to a Pandas Series
pd_series = pd.Series(review_lengths)

# Plotting the distribution of reviewText_length using Pandas/Matplotlib
plt.figure(figsize=(10, 6))
pd_series.hist(bins=50)  # Adjust the number of bins for your specific dataset
plt.title('Distribution of Review Text Lengths')
plt.xlabel('Review Text Length')
plt.ylabel('Count')
plt.grid(False)
plt.show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `reviewText_length` cannot be resolved. Did you mean one of the following? [`reviewText`, `reviewTime`, `reviewerName`, `reviewerID`, `overall`].;
'Project ['reviewText_length]
+- Relation [asin#1527,image#1528,overall#1529,reviewText#1530,reviewTime#1531,reviewerID#1532,reviewerName#1533,style#1534,summary#1535,unixReviewTime#1536L,verified#1537,vote#1538] json


In [76]:
# Example of filtering based on a condition, such as reviews that are too short/long might be outliers
df = df.filter(col('reviewText_length') > 10)  # Example threshold
df = df.filter(col('reviewText_length') < 1000)  # Example threshold

## Text Processing

In [77]:
# Text Processing - cleaning the review text, undercase the words, tokenizing, and removing stop words (simplified example)
from pyspark.ml.feature import Tokenizer, StopWordsRemover

df = df.withColumn('reviewText', lower(col('reviewText'))) 
tokenizer = Tokenizer(inputCol='reviewText', outputCol='reviewText_tokens')
df = tokenizer.transform(df)
remover = StopWordsRemover(inputCol='reviewText_tokens', outputCol='reviewText_clean')
df = remover.transform(df)

IllegalArgumentException: Output column reviewText_tokens already exists.

In [78]:
# Show the processed DataFrame
df.show(10)

+----------+-------+--------------------+--------------+--------------------+--------------------+--------------+--------+-----------------+--------------------+--------------------+---+-----+----+
|      asin|overall|          reviewText|    reviewerID|        reviewerName|             summary|unixReviewTime|verified|reviewText_length|   reviewText_tokens|    reviewText_clean|Day|Month|Year|
+----------+-------+--------------------+--------------+--------------------+--------------------+--------------+--------+-----------------+--------------------+--------------------+---+-----+----+
|B0000223SK|    2.0|looking for sand ...| AQPJCQ3F7XEH1|         chuck Wagon|Not for Porter-Ca...|    1447545600|    true|              419|[looking, for, sa...|[looking, sand, p...| 11|   15|2015|
|B0000223UV|    5.0|this is the best ...|A10ZZC3W6LJPQK|Adrian Aarons GUT...|This is the best ...|    1486684800|    true|              319|[this, is, the, b...|[best, way, repai...| 02|   10|2017|
|B00002NCE

In [73]:

# Split reviewTime into day, month, and year
split_col = split(df['reviewTime'], ' ')
df = df.withColumn('Day', split_col.getItem(0))
df = df.withColumn('Month', split_col.getItem(1).substr(0,2))
df = df.withColumn('Year', split_col.getItem(2))
df = df.drop('reviewTime')

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `reviewTime` cannot be resolved. Did you mean one of the following? [`asin`, `overall`, `reviewText`, `reviewerID`, `reviewerName`, `summary`, `unixReviewTime`, `verified`, `reviewText_length`, `reviewText_tokens`, `reviewText_clean`, `Day`, `Month`, `Year`].

In [79]:
df.select('reviewerName').distinct().count()

3714

In [80]:
df.select('asin').distinct().count()

1902

In [81]:
from pyspark.sql.functions import avg, count

# Average rating
average_rating = df.agg(avg("overall")).first()[0]

# Count of reviews for each asin
reviews_per_asin = df.groupBy("asin").agg(count("reviewerID").alias("Number_of_Reviews"))

In [82]:
# Count of reviews for each reviewer
reviews_per_reviewers = df.groupBy("reviewerID").agg(count("asin").alias("Number_of_Reviews"))

In [84]:

# Convert Spark DataFrame to Pandas for Visualization
pdf = df.toPandas()
reviews_per_asin_pdf = reviews_per_asin.toPandas()

# Histogram for overall ratings
plt.figure(figsize=(10, 6))
pdf['overall'].hist()
plt.title('Distribution of Overall Ratings')
plt.xlabel('Rating')
plt.ylabel('Count')
plt.grid(False)
plt.show()

# Plotting the pie chart for overall ratings
rating_counts = pdf['overall'].value_counts().sort_index()
plt.figure(figsize=(10, 6))
plt.pie(rating_counts, labels=rating_counts.index, autopct='%1.1f%%', startangle=140)
plt.title('Distribution of Overall Ratings')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()




23/11/22 13:24:35 ERROR RetryingBlockTransferor: Exception while beginning fetch of 1 outstanding blocks 
java.io.IOException: Failed to connect to /172.20.10.3:55898
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:294)
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:214)
	at org.apache.spark.network.netty.NettyBlockTransferService$$anon$2.createAndStart(NettyBlockTransferService.scala:131)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.transferAllOutstanding(RetryingBlockTransferor.java:173)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.start(RetryingBlockTransferor.java:152)
	at org.apache.spark.network.netty.NettyBlockTransferService.fetchBlocks(NettyBlockTransferService.scala:151)
	at org.apache.spark.network.BlockTransferService.fetchBlockSync(BlockTransferService.scala:102)
	at org.apache.spark.storage.BlockManager.fetchRemoteManagedBuffer(BlockMana

Py4JJavaError: An error occurred while calling o364.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 225.0 failed 1 times, most recent failure: Lost task 0.0 in stage 225.0 (TID 154) (172.20.10.3 executor driver): TaskResultLost (result lost from block manager)
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2844)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2780)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2779)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2779)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$

In [ ]:

# Bar chart for number of reviews for each asin
top_reviews_per_asin_pdf = reviews_per_asin_pdf.sort_values(by='Number_of_Reviews', ascending=False).head(100)
plt.figure(figsize=(10, 6))
# Set the 'asin' as the index and plot the 'review_count' column
top_reviews_per_asin_pdf.set_index('asin')['Number_of_Reviews'].plot(kind='bar', legend=False)
plt.title('Number of Reviews for Top 100 ASINs')
plt.ylabel('Count')
plt.xlabel('ASIN')
plt.grid(False)
plt.tight_layout()
plt.show()

In [ ]:
# Bar chart for number of reviews for each reviewers
reviews_per_reviewers_pdf = reviews_per_reviewers.toPandas()

top_reviews_per_reviewers_pdf = reviews_per_reviewers_pdf.sort_values(by='Number_of_Reviews', ascending=False).head(100)
plt.figure(figsize=(10, 6))
# Set the 'reviewers' as the index and plot the 'review_count' column
top_reviews_per_reviewers_pdf.set_index('reviewerID')['Number_of_Reviews'].plot(kind='bar', legend=False)
plt.title('Number of Reviews for Top 100 reviewers')
plt.ylabel('Count')
plt.xlabel('reviewers')
plt.grid(False)
plt.tight_layout()
plt.show()

In [ ]:
# Convert reviewText and summary columns to a single string
review_text_str = ' '.join(df.rdd.map(lambda row: row.reviewText).collect())
summary_str = ' '.join(df.rdd.map(lambda row: row.summary).collect())

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def generate_word_cloud(text, title):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(title)
    plt.show()

generate_word_cloud(review_text_str, "Word Cloud for Review Text")
generate_word_cloud(summary_str, "Word Cloud for Summary")


# Modelling

## Data Splitting

In [ ]:
from pyspark.ml.feature import StringIndexer

# Indexing is required to convert string identifiers to numeric indices for the ALS algorithm
indexer_user = StringIndexer(inputCol="reviewerID", outputCol="userIndex")
df = indexer_user.fit(df).transform(df)

indexer_item = StringIndexer(inputCol="asin", outputCol="itemIndex")
df = indexer_item.fit(df).transform(df)

In [ ]:
(train, test) = df.randomSplit([0.8, 0.2])


## ALS

refer: https://www.kaggle.com/code/nadianizam/h-m-fashion-recommendation-with-pyspark



In [ ]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

# Setting up the ALS model
als = ALS(maxIter=5, regParam=0.01, userCol="userIndex", itemCol="itemIndex", ratingCol="overall", coldStartStrategy="drop")

# Fitting the ALS model on the training data
model = als.fit(train)

# Predicting on the train and test sets
predictions_train = model.transform(train)
predictions_test = model.transform(test)

# Create an RMSE evaluator using the label and predicted columns
reg_evaluator = RegressionEvaluator(metricName="rmse", labelCol="overall", predictionCol="prediction")

# Evaluate the model on training data
rmse_train = reg_evaluator.evaluate(predictions_train)

# Evaluate the model on test data
rmse_test = reg_evaluator.evaluate(predictions_test)


In [ ]:
model.recommendForAllUsers(10).show(1, truncate = False)
model.save("models/RatingALSModel.obj")

In [ ]:
rmse_train

In [ ]:
rmse_test

## Recall

In [ ]:
from pyspark.ml.recommendation import ALSModel
recommendnum = 10
als_model = ALSModel.load("models/RatingALSModel.obj")
als_model.recommendForAllUsers(recommendnum).show(1)

In [ ]:
train.groupBy("userIndex").count().count()

In [ ]:
train.filter(train['userIndex'] == testUserID).select('itemIndex').show(10)

In [ ]:
train.select('userIndex').show(1)

In [ ]:
df_temp = als_model.recommendForAllUsers(10)

In [ ]:
x = df_temp.select('recommendations').collect()

In [ ]:
from pyspark.sql.functions import col,explode
df_temp0 = df_temp.withColumn("name", explode(col('recommendations')))
df_temp0.withColumn("name2", col('name').getItem(0)).show(10)

In [ ]:
from pyspark.sql.functions import col,expr
df_temp0 = df_temp.withColumn("name", expr("transform(recommendations, r -> r.itemIndex)"))
df_temp0.show(10)

In [ ]:
train.filter(train['userIndex'] == m.userIndex).select('itemIndex')

In [ ]:
from pyspark.sql.functions import collect_list
actualitem = train.groupby('userIndex').agg(collect_list('itemIndex').alias('indexlist'))

In [ ]:
result = df_temp0.join(actualitem, 'userIndex')

In [ ]:
from pyspark.sql.functions import array_intersect, size, avg
metrics = result.withColumn('TP', size(array_intersect(col('name'),col('indexlist')))) \
                .withColumn('precision', col('TP')/recommendnum) \
                .withColumn('recall', col('TP')/size(col('indexlist'))) \
                .withColumn('F1', when((col('precision') + col('recall')) >0,
                                        2* (col('precision') *col('recall'))/(col('precision') +col('recall')))
                             .otherwise(0))


In [ ]:
from pyspark.sql.functions import max as ma
average_metrics = metrics.agg(
    ma(col('precision')),
    ma(col('recall')),
    ma(col('F1'))
)
average_metrics.show()

In [ ]:
metrics.show()

In [ ]:
df_temp0.withColumn("name2", col('name').getItem(0))

In [ ]:
for r in als_model.userFactors.select("id").collect():
    userId = r.id
    cateId_df = pd.DataFrame(pdf.cateId,unique(),columns=['cateId'])
    cateId_df.insert(0,'userId',np.array([userId for i in range(6769)]))

    ret = set()
    # 利用模型，传入datasets(userId, cateId)，这里控制了userId一样，所以相当于是在求某用户对所有分类的兴趣程度
    cateId_list = als_model.transform(spark.createDataFrame(cateId_df)).sort('prediction',ascending=False).na.drop()
   
    # 从前20个分类中选出500个进行召回
    for i in cateId_list.head(20):
        need = 500 - len(ret)    # 如果不足500个，那么随机选出need个广告
        ret = ret.union(np.random.choice(pdf.where(pdf.cateId==i.cateId).adgroupId.dropna().astype(np.int64),need))
        if len(ret) >= 500:    # 如果达到500个则退出
            break
    client.sadd(userId, *ret)     

## GraphFrame

In [ ]:
!pip install graphframes

In [ ]:
from graphframes import GraphFrame

# Assuming 'df' is your original PySpark DataFrame and it has been indexed
# Create vertices DataFrame
vertices = df.selectExpr("userIndex as id").distinct().union(df.selectExpr("itemIndex as id").distinct()) # check if those two index overlap

# Create edges DataFrame
edges = df.selectExpr("userIndex as src", "itemIndex as dst", "overall as rating")

# Create a GraphFrame
graph = GraphFrame(vertices, edges)

In [ ]:
# TO DO
# 1: Hyperparameter tuning
# 2: How to make use of the graph functions?

# Meta Data

In [ ]:
df_meta.printSchema()

In [ ]:
print("item总数：", df_meta.groupBy("asin").count().count())
print("brand总数：", df_meta.groupBy("brand").count().count())
print("category总数：", df_meta.groupBy("category").count().count())

In [ ]:
print("分类特征值个数情况: ")
print("cms_segid: ", user_profile_df.groupBy("cms_segid").count().count())
print("cms_group_id: ", user_profile_df.groupBy("cms_group_id").count().count())
print("final_gender_code: ", user_profile_df.groupBy("final_gender_code").count().count())
print("age_level: ", user_profile_df.groupBy("age_level").count().count())
print("shopping_level: ", user_profile_df.groupBy("shopping_level").count().count())
print("occupation: ", user_profile_df.groupBy("occupation").count().count())

print("含缺失值的特征情况: ")
df_meta.groupBy("details").count().show()
df_meta.groupBy("feature").count().show()
df_meta.groupBy("fit").count().show()

t_count = df_meta.count()

pl_na_count = t_count - user_profile_df.dropna(subset=["pvalue_level"]).count()
print("pvalue_level的空值情况：", pl_na_count, "空值占比：%0.2f%%"%(pl_na_count/t_count*100))

nul_na_count = t_count - user_profile_df.dropna(subset=["new_user_class_level"]).count()
print("new_user_class_level的空值情况：", nul_na_count, "空值占比：%0.2f%%"%(nul_na_count/t_count*100))


In [ ]:
print("含缺失值的特征情况: ")
df_meta.groupBy("details").count().show()
df_meta.groupBy("feature").count().show()
df_meta.groupBy("fit").count().show()